# HateXplain

https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two


The model is used for classifying a text as Abusive (Hatespeech and Offensive) or Normal. The model is trained using data from Gab and Twitter and Human Rationales were included as part of the training data to boost the performance. The model also has a rationale predictor head that can predict the rationales given an abusive sentence.

The dataset and models are available here: https://github.com/punyajoy/HateXplain

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BatchEncoding
### from models.py
from models import *
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf

tf.get_logger().setLevel('ERROR')

seed = 42
batch_size = 10
test_size = 30
MAX_LENGTH = 128

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

In [ ]:
# loading in testing data set
customer_data = pd.read_csv("datasets\\Customer support tweets\\twcs.csv").drop(columns=["tweet_id", "author_id", "inbound", "created_at", "response_tweet_id","in_response_to_tweet_id"])

In [ ]:
# Source https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379

def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             padding='longest', #implements dynamic padding
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    

In [ ]:
data = customer_data.text.sample(test_size).values.tolist()
input_ids = []
attention_mask = []


batchencoder = (tokenizer(data[0:10], return_tensors="pt", padding=True))

print(batchencoder['input_ids'])
#                        padding='longest', #implements dynamic padding
#                        return_attention_mask=True,
#                        return_token_type_ids=False).convert_to_tensors("tf")

# input_ids = batchencoder.convert_to_tensors("tf")
# max_length = np.max(list(map(len, data)))
# print(max_length)
# input_ids, attention_mask = batch_encode(tokenizer=tokenizer, texts=data, max_length=max_length)
# print(input_ids)

In [ ]:
prediction_logits, _ = model(input_ids=batchencoder['input_ids'],attention_mask=batchencoder['attention_mask'])

In [ ]:
torch.argmax(prediction_logits, dim=1)